# Actions

In [1]:
import os
import re
import csv
from urllib import urlopen
import sys
import csv
from collections import Counter
from __future__ import division

csv.field_size_limit(sys.maxsize)

131072

In [4]:
# load data
recs = []
dir = '.'
with open("../Data/upr-info.csv",'rU') as csvfile:
            reader = csv.DictReader(csvfile)
            for row in reader:
                recs.append(row)
            
print len(recs) # 42013 
recs[0]

42013


{'Action': '4',
 'From': 'Belgium',
 'From.Org': 'EU, OIF',
 'From.Rgrp': 'WEOG',
 'From_COW': 'BEL',
 'Institution': '[]',
 'Issue': 'Freedom of religion and belief',
 'Response': 'Noted',
 'Session': '1',
 'Text': 'Respect freedom of religion and belief, revise the text of the 2006 Ordinance, and suspend its application in the meantime.',
 'To': 'Algeria',
 'To.Org': 'AU, OIC, AL',
 'To.Rgrp': 'Africa',
 'To_COW': 'ALG',
 'UID': 'ALG-BEL-1-1'}

In [5]:
recs[1].keys()

['From_COW',
 'From',
 'To.Org',
 'Text',
 'To.Rgrp',
 'UID',
 'Response',
 'From.Org',
 'To',
 'Session',
 'Action',
 'From.Rgrp',
 'Issue',
 'Institution',
 'To_COW']

In [6]:
# strip extra whitespace on everything
for i in recs:
    i['Text']=' '.join(i['Text'].split())

In [7]:
# remove stop words function
stopwords = ['of','on','all','the','a','for','to','and','to','against','from','in','or']

def noStop(in_text):
    '''
    this function inputs a text, lowers it, and removes stop words
    '''
    text = in_text.lower().split()
    for word in stopwords:
        text = [x for x in text if x != word]
    return(' '.join(text))

## Make Action Dictionary

In [8]:
# assign action
keywords = {}
keywords['ratify'] = ['ratif']
keywords['accede'] = ['acced']
keywords['sign'] = ['sign ']
keywords['implement'] = ['implement']
keywords['establish'] = ['establish']
keywords['continue'] = ['continue']
keywords['take-measures'] = ['take measures']
keywords['harmonize'] = ['harmoniz']
keywords['report'] = ['report','reporting','reports']
keywords['party'] = ['party']
keywords['withdraw'] = ['withdraw']
keywords['cooperate'] = ['cooperat']
keywords['share'] = ['share', 'sharing']

print (keywords.keys())


['accede', 'ratify', 'take-measures', 'share', 'sign', 'report', 'continue', 'withdraw', 'harmonize', 'party', 'implement', 'cooperate', 'establish']


In [9]:
for i in recs:
    i['Action_Verb'] = []
    for key in keywords.keys():
        for word in keywords[key]:
            if key in i['Action_Verb']: # skip if it's already in there
                continue
            if word in noStop(i['Text']):
                i['Action_Verb'].append(key)

In [10]:
# special rules 

# international assistance
for i in recs:
    if (
        ('assistance ' in i['Text'] or 'support ' in i['Text']) and
        'international' in i['Text']
        ):
        i['Action_Verb'].append('assistance')
        

# take ____ measures:
for i in recs:
    if 'take-measures' not in i['Action_Verb']:
        text = i['Text'].lower().replace(',','').split()
        if 'take' in text:
            takeIndex = text.index('take')
            if 'measures' in text[takeIndex:takeIndex+10]:
                i['Action_Verb'].append('take-measures')
        elif 'taking' in text:
            takeIndex = text.index('taking')
            if 'measures' in text[takeIndex:takeIndex+10]:
                i['Action_Verb'].append('take-measures')

In [11]:
# count the actions
actions = []
for rec in recs:
    for i in rec['Action_Verb']:
        actions.append(i)
Counter(actions)

Counter({'continue': 7377, 'ratify': 4978, 'implement': 4440, 'take-measures': 2637, 'establish': 2474, 'cooperate': 1417, 'report': 929, 'accede': 855, 'sign': 592, 'assistance': 587, 'party': 537, 'share': 287, 'withdraw': 279, 'harmonize': 234})

## WRITE CSV

In [12]:
#writing column headings
keys = recs[0].keys()
keys

['From_COW',
 'Action_Verb',
 'From',
 'To.Org',
 'Text',
 'To.Rgrp',
 'UID',
 'Response',
 'From.Org',
 'To',
 'Session',
 'Action',
 'From.Rgrp',
 'Issue',
 'Institution',
 'To_COW']

In [13]:
#writing the rest
with open('../Data/upr-info.csv', 'wb') as output_file:
    dict_writer = csv.DictWriter(output_file, keys)
    dict_writer.writeheader()
    dict_writer.writerows(recs)